# 04 Inference for Time Series (HAC/Newey-West)

Why naive SE break; HAC SE and interpretation.


## Table of Contents
- [Assumptions](#assumptions)
- [Autocorrelation](#autocorrelation)
- [HAC SE](#hac-se)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Regression is the bridge between statistics and ML. You will learn:
- single-factor vs multi-factor interpretation,
- robust standard errors,
- coefficient stability and multicollinearity.


## Prerequisites (Quick Self-Check)
- Completed Parts 00–01 (foundations + data).
- Basic algebra comfort (reading coefficient tables, units).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Treating coefficients as causal without a causal design.
- Ignoring multicollinearity (unstable coefficients).

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/02_regression/04_inference_time_series_hac.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/02_regression/04_inference_time_series_hac.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Learn why naive OLS standard errors often break in macro time series, and how HAC/Newey-West helps.

Big idea:
- coefficients answer "what best fits the line"
- standard errors answer "how uncertain are we about the coefficients"

Time series often violate the assumptions behind naive SE.



## Primer: `statsmodels` vs `scikit-learn` (inference vs prediction)

This repo uses both libraries because they serve different goals:

- **Prediction (ML):** optimize out-of-sample accuracy → `scikit-learn`
- **Inference (econometrics):** interpret coefficients + quantify uncertainty → `statsmodels`

### Minimal `statsmodels` OLS pattern

```python
import statsmodels.api as sm

# X: DataFrame of features, y: Series target
Xc = sm.add_constant(X, has_constant="add")  # add intercept
res = sm.OLS(y, Xc).fit()
print(res.summary())
```

**Expected output / sanity check**
- a table with `coef`, `std err`, `t`, `P>|t|`, and a CI column
- coefficient names match your column names

### What you are looking at in `res.summary()`

- **coef**: $\\hat\\beta$ (estimated effect in the model)
- **std err**: estimated uncertainty $\\widehat{SE}(\\hat\\beta)$
- **t**: $\\hat\\beta / \\widehat{SE}(\\hat\\beta)$
- **P>|t|**: p-value for $H_0: \\beta=0$ (conditional on assumptions)
- **[0.025, 0.975]**: 95% confidence interval

### Robust standard errors (change uncertainty, not coefficients)

```python
# Cross-section heteroskedasticity
res_hc3 = res.get_robustcov_results(cov_type="HC3")

# Time series autocorrelation + heteroskedasticity
res_hac = res.get_robustcov_results(cov_type="HAC", cov_kwds={"maxlags": 4})
```

### Common pitfalls (and quick fixes)

- **Forgetting the intercept**
  - Fix: always `add_constant`.
- **Wrong SE for time series**
  - Fix: use HAC when residuals are autocorrelated.
- **Treating p-values as causal proof**
  - Fix: write the identification assumption; otherwise interpret as association.
- **Mixing prediction and inference**
  - Fix: use `sklearn` pipelines + time splits for prediction; use `statsmodels` for coefficient uncertainty.


## Primer: Hypothesis testing (p-values, t-stats, confidence intervals)

You will see p-values, t-statistics, and confidence intervals in regression output (especially `statsmodels`). This primer gives you the minimum to interpret them correctly.

### The objects (plain language)

- **Null hypothesis** $H_0$: the default claim (often “no effect”).
- **Alternative** $H_1$: the claim you consider if the data looks inconsistent with $H_0$.
- **Test statistic**: “how far” your estimate is from the null, in uncertainty units.
- **p-value**: probability (under the null *and model assumptions*) of seeing a test statistic at least as extreme as observed.
- **Confidence interval (CI)**: a range of parameter values consistent with the data under assumptions.

### What a p-value is NOT

- Not the probability $H_0$ is true.
- Not the probability the model is correct.
- Not a measure of economic importance.

### Regression t-test intuition

In OLS, a common test is $H_0: \\beta_j = 0$.

$$
t_j = \\frac{\\hat\\beta_j}{\\widehat{SE}(\\hat\\beta_j)}
$$

If you change your SE estimator (HC3/HAC/cluster), you change $\\widehat{SE}$ and therefore the p-value, even if the coefficient stays the same.

### Expected output / what you should look at in `res.summary()`

- `coef`: effect size (in model units)
- `std err`: uncertainty
- CI columns: magnitude + uncertainty together

### Common pitfalls in this project

- Macro time series often have autocorrelation → naive SE too small → use HAC when interpreting p-values.
- Multiple testing/spec-search can produce small p-values by chance.
- Predictive success ≠ causal interpretation.

### Tiny demo (toy; not project data)

```python
import numpy as np
import pandas as pd
import statsmodels.api as sm

rng = np.random.default_rng(0)
n = 300
x = rng.normal(size=n)
y = 1.0 + 0.5 * x + rng.normal(scale=1.0, size=n)

df = pd.DataFrame({"y": y, "x": x})
X = sm.add_constant(df[["x"]])
res = sm.OLS(df["y"], X).fit()
print(res.summary())
```


<a id="assumptions"></a>
## Assumptions

### Goal
Fit a baseline regression and inspect residuals.

Reminder:
- OLS coefficients can be computed even when assumptions fail.
- Inference (SE / p-values / CI) is what becomes unreliable.



### Your Turn (1): Load macro data and pick a simple regression


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'macro_quarterly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)

y_col = 'gdp_growth_qoq'
x_cols = ['T10Y2Y_lag1']

df_m = df[[y_col] + x_cols].dropna().copy()
df_m.tail()



### Your Turn (2): Fit OLS and compute residuals


In [ ]:
import statsmodels.api as sm

X = sm.add_constant(df_m[x_cols], has_constant='add')
y = df_m[y_col]
res = sm.OLS(y, X).fit()

# Residuals = y - y_hat
resid = res.resid
fitted = res.fittedvalues

print(res.summary())
resid.head()



### Your Turn (3): Residual diagnostics plots (simple but useful)


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot residuals over time
# TODO: Plot residuals vs fitted values (heteroskedasticity visual check)
...



<a id="autocorrelation"></a>
## Autocorrelation

### Goal
Measure whether residuals are correlated over time.

If residuals are autocorrelated, naive SE can be too small.



### Your Turn (1): Autocorrelation by lag


In [ ]:
# TODO: Compute residual autocorrelation at lags 1..8.
# Hint: resid.autocorr(lag=k)
ac = {k: float(resid.autocorr(lag=k)) for k in range(1, 9)}
ac



### Your Turn (2): A simple ACF plot


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot the autocorrelation values as a bar chart.
...



<a id="hac-se"></a>
## HAC SE

### Goal
Fit the same model but compute HAC/Newey-West robust SE.

Your task:
- compare coefficient (should be the same)
- compare SE/p-values (can differ)



### Your Turn (1): Compare naive vs HAC with different maxlags


In [ ]:
from src import econometrics

res_naive = econometrics.fit_ols(df_m, y_col=y_col, x_cols=x_cols)
res_hac1 = econometrics.fit_ols_hac(df_m, y_col=y_col, x_cols=x_cols, maxlags=1)
res_hac4 = econometrics.fit_ols_hac(df_m, y_col=y_col, x_cols=x_cols, maxlags=4)

# TODO: Print SE/p-values side-by-side for the slope coefficient.
...



### Your Turn (2): Write a careful interpretation


In [ ]:
# TODO: Write 6-10 sentences answering:
# - Did HAC increase or decrease your SE?
# - How did that affect your p-value/CI?
# - What assumptions are still required even with HAC?
notes = """
...
"""
print(notes)



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Assumptions</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_inference_time_series_hac — Assumptions
# OLS assumptions (esp. independent errors) often fail in time series.
```

</details>

<details><summary>Solution: Autocorrelation</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_inference_time_series_hac — Autocorrelation
import pandas as pd
import numpy as np

df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)
y = df['gdp_growth_qoq'].to_numpy()
print('lag1 autocorr (rough):', np.corrcoef(y[1:], y[:-1])[0,1])
```

</details>

<details><summary>Solution: HAC SE</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_inference_time_series_hac — HAC SE
from src import econometrics

res = econometrics.fit_ols(df, y_col='gdp_growth_qoq', x_cols=['T10Y2Y'])
res_hac = econometrics.fit_ols_hac(df, y_col='gdp_growth_qoq', x_cols=['T10Y2Y'], maxlags=2)
print('naive SE:', res.bse)
print('HAC SE  :', res_hac.bse)
```

</details>

